## Initialization

Import findspark and initialize, then import pyspark and sparkSession and create a Spark Session

In [ ]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("<spark url>") \
                            .appName("GBDT") \
                            .config("spark.executor.instances", "1") \
                            .config("spark.executor.cores", "2") \
                            .config("spark.executor.memory", "1536m") \
                            .getOrCreate()

Create schema and Dataframe

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType,\
                                FloatType, LongType, DecimalType
schema = StructType([ \
                     StructField('index', StringType(), True), \
                     StructField('app_id', LongType(), True), \
                     StructField('app_name', StringType(), True), \
                     StructField('review_id', LongType(), True), \
                     StructField('language', StringType(), True), \
                     StructField('review', StringType(), True), \
                     StructField('timestamp_created', LongType(), True), \
                     StructField('timestamp_updated', LongType(), True), \
                     StructField('recommended', BooleanType(), True), \
                     StructField('votes_helpful', IntegerType(), True), \
                     StructField('votes_funny', IntegerType(), True), \
                     StructField('weighted_vote_score', FloatType(), True), \
                     StructField('comment_count', IntegerType(), True), \
                     StructField('steam_purchase', BooleanType(), True), \
                     StructField('received_for_free', BooleanType(), True), \
                     StructField('written_during_early_access', BooleanType(), True), \
                     StructField('author_steamid', LongType(), True), \
                     StructField('author_num_games_owned', IntegerType(), True), \
                     StructField('author_num_reviews', IntegerType(), True), \
                     StructField('author_playtime_forever', DecimalType(), True), \
                     StructField('author_playtime_last_two_weeks', DecimalType(), True), \
                     StructField('author_playtime_at_review', DecimalType(), True), \
                     StructField('author_last_played', FloatType(), True), \
                    ])

In [ ]:
from pyspark.sql.functions import col

df = spark.read.csv("steam_reviews.csv", header = True, schema = schema, multiLine = True,\
                    lineSep = "\r")
df = df.drop("review", "index", "app_name", "language")
df = df.withColumn("recommended",col("recommended").cast(IntegerType()))
df = df.na.drop()

Prepare data for Classification

In [ ]:
from pyspark.ml.feature import VectorAssembler

cols = df.columns
cols.remove("recommended")
assembler = VectorAssembler(inputCols = cols, outputCol = "features")

data = assembler.transform(df)
data = data.select("features", "recommended")
data.show(truncate = False)

In [ ]:
train, test = data.randomSplit([0.9, 0.1])

In [ ]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import time

t = time.time()

gbt = GBTClassifier(labelCol = "recommended", featuresCol = "features")
train.cache()
model = gbt.fit(train)
prediction_test = model.transform(test)
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='recommended',\
                                          metricName='accuracy')
evaluator.evaluate(prediction_test)
duration = (time.time() - t)
train.unpersist()
duration